## Sinusoidal curve fitting

In [1]:
import numpy as np
import scipy.stats
import os
import glob
import pandas as pd
import numpy as np
import pyarrow as pa
from tqdm import tqdm
from dbfread import DBF
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import scipy.stats as stats
from scipy.stats import ks_2samp
from scipy.optimize import minimize
from matplotlib.dates import MonthLocator, DateFormatter
import warnings
import multiprocessing
from lmfit import Model
from lmfit import minimize,Parameters,Parameter,report_fit

print('loaded')

loaded


In [3]:
def filteredComidTo60pcMedian(one_comid):
    one_comid_copy = one_comid.copy()
    min_data_taken_from_high_freq_stn = 0.6

    # Make proper date format
    one_comid_copy['date'] = pd.to_datetime(one_comid_copy['date'])
    base_date = pd.to_datetime('2000-01-01')
    one_comid_copy['days2000'] = (one_comid_copy['date'] - base_date).dt.days
    one_comid_copy.drop('date', axis=1, inplace=True)
    
    # Limit the data by taking 60% data from top n frequent COMID
    river_id_counts = one_comid_copy['riverID'].value_counts()
    cumulative_sum = river_id_counts.cumsum()
    total_sum = river_id_counts.sum()
    threshold = min_data_taken_from_high_freq_stn * total_sum
    filtered_data = one_comid_copy[one_comid_copy['riverID'].isin(river_id_counts.index[0:cumulative_sum[cumulative_sum < threshold].count() + 1])]
    
    filtered_data['width_norm'] = filtered_data['width'] * filtered_data['width'].mean() / filtered_data.groupby('riverID')['width'].transform('mean')
    
    # Limit the data to 2% and 98% quantile width_norm
    quantile_bot = filtered_data['width_norm'].quantile(0.02)
    quantile_top = filtered_data['width_norm'].quantile(0.98)
    filtered_data = filtered_data[(filtered_data['width_norm'] > quantile_bot) & (filtered_data['width_norm'] < quantile_top)]
    
    # Median data computation
    median_days2000 = filtered_data.groupby('days2000')['width_norm'].median()
    return median_days2000

# Standard Error of Means
def calculate_sem_from_residuals(residuals):
    std_dev = np.std(residuals)
    n = len(residuals)
    sem = std_dev / np.sqrt(n)
    return sem

# Root Relative Squared Error (RRSE)
def calculate_RRSE(original_data, projected_data):
    mean_original = np.mean(original_data)
    numerator = np.sum((projected_data - original_data) ** 2)
    denominator = np.sum((mean_original - original_data) ** 2)
    return np.sqrt(numerator / denominator)

def perform_kuipers_test(data1, data2):
    # Perform the Kuipers test for two datasets
    ks_statistic = ks_2samp(data1, data2).statistic
    # A great fit has a low KS statistic, indicating similarity between the two datasets.
    # A bad fit has a high KS statistic, indicating dissimilarity between the two datasets.
    return ks_statistic

# Relative Absolute Error (RAE)
def calculate_RAE(original_data, projected_data):
    mean_original = np.mean(original_data)
    numerator = np.sum(np.abs(projected_data - original_data))
    denominator = np.sum(np.abs(mean_original - original_data))
    return numerator / denominator

# Normalized Root Mean Square Error (NRMSE)
def calculate_NRMSE(original_data, projected_data):
    rmse = np.sqrt(np.mean((projected_data - original_data) ** 2))
    data_range = np.max(original_data) - np.min(original_data)
    nrmse = rmse / data_range
    return nrmse

# Define the sine function with a custom time period
def sine_curve_one(x, amplitude, intercept, phase):
    time_period = 365.25
    return np.abs(float(amplitude)) * np.sin((2 * np.pi * x / time_period) + (2 * np.pi * phase / time_period)) + intercept

def calculate_cv_iqr(data):
    mean = np.mean(data)
    std_dev = np.std(data)
    cv = (std_dev / mean) * 100

    q1, q3 = np.percentile(data, [25, 75])
    iqr = q3 - q1

    return cv, iqr, len(data)

In [4]:
def process_COMID(thatCOMID, thatCOMID_sampling, COMID_dataCount):
    mean_months = thatCOMID_sampling.groupby('month')['width'].mean()
    unique_months = len(mean_months)
    cv_month, iqr_month, count_month = calculate_cv_iqr(mean_months.values)

    median_days2000 = filteredComidTo60pcMedian(thatCOMID_sampling)
    residuals_flat = median_days2000 - median_days2000.mean()
    COMID_dataCount[thatCOMID] =  len(median_days2000)

    if len(median_days2000) > 60:
        standard_error = calculate_sem_from_residuals(residuals_flat)
        cv = stats.variation(median_days2000.values) * 100

        dates = median_days2000.index
        widths = median_days2000.values
        stddev = np.std(widths)

        x = np.asarray(dates)
        y = np.asarray(widths)

        # Define multiple sets of parameters with different phase guesses
        params0 = Parameters()
        params0.add('amplitude', value=stddev * 5, min=0)
        params0.add('intercept', value=np.median(y) / 2, min=0)
        params0.add('phase', value=0, min=-365.25 * 2, max=365.25 * 3)

        params90 = Parameters()
        params90.add('amplitude', value=stddev * 5, min=0)
        params90.add('intercept', value=np.median(y) / 2, min=0)
        params90.add('phase', value=90, min=-365.25 * 2, max=365.25 * 3)

        params180 = Parameters()
        params180.add('amplitude', value=stddev * 5, min=0)
        params180.add('intercept', value=np.median(y) / 2, min=0)
        params180.add('phase', value=180, min=-365.25 * 2, max=365.25 * 3)

        params270 = Parameters()
        params270.add('amplitude', value=stddev * 5, min=0)
        params270.add('intercept', value=np.median(y) / 2, min=0)
        params270.add('phase', value=270, min=-365.25 * 2, max=365.25 * 3)

        # Create models with different phase guesses
        dmodel0 = Model(sine_curve_one)
        result0 = dmodel0.fit(y, params0, x=x)
        
        dmodel90 = Model(sine_curve_one)
        result90 = dmodel90.fit(y, params90, x=x)
        
        dmodel180 = Model(sine_curve_one)
        result180 = dmodel180.fit(y, params180, x=x)

        dmodel270 = Model(sine_curve_one)
        result270 = dmodel270.fit(y, params270, x=x)
        
        # Calculate the sum of squared residuals for each fit
        ssr0 = np.sum(result0.residual ** 2)
        ssr90 = np.sum(result90.residual ** 2)
        ssr180 = np.sum(result180.residual ** 2)
        ssr270 = np.sum(result270.residual ** 2)

        # Choose the best fit based on the least sum of squares (LSS)
        best_result = min((result0, result90, result180, result270), key=lambda result: np.sum(result.residual ** 2))

        amp = best_result.params['amplitude'].value
        inc = best_result.params['intercept'].value
        pek = best_result.params['phase'].value

        params = (amp, inc, pek)
        sine_curve_one_gen_data = sine_curve_one(dates, *params)
        residuals_curve = median_days2000.values - sine_curve_one_gen_data
        standard_error_sine = calculate_sem_from_residuals(residuals_curve)

        slope, intercept, r_value, p_value, std_err = stats.linregress(median_days2000.index, median_days2000.values)

        iqr = np.percentile(widths, 75) - np.percentile(widths, 25)
        median_widths = np.median(widths)

        ks_stat2 = perform_kuipers_test(widths, sine_curve_one_gen_data)
        rrse = calculate_RRSE(widths, sine_curve_one_gen_data)
        rae = calculate_RAE(widths, sine_curve_one_gen_data)
        nrmse = calculate_NRMSE(widths, sine_curve_one_gen_data)

        return [thatCOMID, amp, inc, pek, cv, slope, intercept, r_value ** 2, p_value, standard_error, standard_error_sine, ks_stat2, rae, rrse, nrmse, stddev, iqr, median_widths, cv_month, iqr_month, count_month]
    else:
        return []



def worker(args):
    return process_COMID(*args)

In [5]:
import requests

def send_ntfy_notification(title, message, priority="urgent", tags="bell,fire"):
    url = "https://ntfy.sh/river_width_res"
    headers = {
        "Title": title,
        "Priority": priority,
        "Tags": tags
    }
    response = requests.post(url, data=message, headers=headers)
    return response

# Example usage after a cell completes
send_ntfy_notification(
    title="Notebook Alert: Cell Execution Complete",
    message="Your cell has successfully completed execution!",
    priority="urgent",
    tags="bell,fire"
)

<Response [200]>

In [2]:
# Load the GLOW-S width with snow affected values removed 
output_path = '/N/lustre/project/proj-212/abhinav/River_Width_analysis/RiverWidthAnalysis/ClimateRegionDivision/HUC_Parquet/allRegion_large_T273check_over60.parquet'
final_large_data_warm_over60 = pd.read_parquet(output_path)

In [3]:
final_large_data_warm_over60

,riverID,date,width,lat,lon,sceneID_unique,COMID,month,date_YMD,hot_enough
0,R12084791XS2389944,2019-01-12 10:09:57,8.448136,-28.952676,27.725176,95294,12084791,1,2019-01-12,1.0
1,R12084791XS2389945,2019-01-12 10:09:57,1.586848,-28.952500,27.724663,95294,12084791,1,2019-01-12,1.0
2,R12084791XS2389944,2019-01-12 10:09:57,10.141748,-28.952676,27.725176,95295,12084791,1,2019-01-12,1.0
3,R12084791XS2389945,2019-01-12 10:09:57,1.586848,-28.952500,27.724663,95295,12084791,1,2019-01-12,1.0
4,R12084791XS2389910,2019-02-06 10:50:10,21.861701,-28.965000,27.723562,95361,12084791,2,2019-02-06,1.0
...,...,...,...,...,...,...,...,...,...,...
1400715000,R78027927XS0147569,2022-12-06 21:50:59,22.856135,42.185966,-124.134868,33477,78027927,12,2022-12-06,1.0
1400715001,R78027927XS0147569,2022-12-13 22:25:00,82.885030,42.185966,-124.134868,33484,78027927,12,2022-12-13,1.0
1400715002,R78027927XS0147570,2022-12-13 22:25:00,34.744213,42.186366,-124.134467,33484,78027927,12,2022-12-13,1.0
1400715003,R78027927XS0147569,2022-12-16 22:22:55,58.222716,42.185966,-124.134868,33498,78027927,12,2022-12-16,1.0


In [4]:
# Remove all the rows with width less than 10m
final_large_data_warm_over60 = final_large_data_warm_over60[final_large_data_warm_over60['width']>10]

In [5]:
final_large_data_warm_over60['riverID'].unique().shape

(23251578,)

In [12]:
# Main code
warnings.filterwarnings("ignore")
index = 0


COMID_dataCount = {}
COMID_dataCollection = [['COMID', 'amplitude', 'intercept', 'phase', 'CV_flat', 'slope', 'intercept_c', 'R2', 'p_value', 'SE_flat', 'SE_sine', 'ks_stat', 'rae', 'rrse', 'nrmse', 'stddev', 'IQR', 'Med_width', 'CV_month', 'IQR_month', 'Count_month']]

jobs = [(thatCOMID, thatCOMID_sampling, COMID_dataCount) for thatCOMID, thatCOMID_sampling in final_large_data_warm_over60.groupby('COMID')]
num_processes = 8

with multiprocessing.Pool(num_processes) as pool:results = list(tqdm(pool.imap(worker, jobs), total=len(jobs)))

100%|████████████████████████████████| 476232/476232 [21:20<00:00, 371.90it/s]


In [13]:
for result in results:
    if result:
        COMID_dataCollection.append(result)

final_large_data_warm_over60_sine_fitting = pd.DataFrame(COMID_dataCollection[1:], columns=COMID_dataCollection[0])

In [14]:
final_large_data_warm_over60_sine_fitting.shape

(342367, 21)

In [15]:
output_path = '/N/lustre/project/proj-212/abhinav/River_Width_analysis/RiverWidthAnalysis/NCDF_out/sine_peak_newresult_below_10m_removed.csv'
final_large_data_warm_over60_sine_fitting.to_csv(output_path, index=False)

print(f"Data saved to {output_path}")

Data saved to /N/lustre/project/proj-212/abhinav/River_Width_analysis/RiverWidthAnalysis/NCDF_out/sine_peak_newresult_below_10m_removed.csv


## IQR analysis

In [16]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed
import multiprocessing
import warnings
warnings.filterwarnings('ignore')

# Define the function that processes each COMID
def process_comid(thatCOMID, thatCOMID_sampling):
    median_days2000 = filteredComidTo60pcMedian(thatCOMID_sampling)
    dates = median_days2000.index
    widths = median_days2000.values

    if median_days2000.shape[0] == 0:
        return [thatCOMID, -1, -1, thatCOMID_sampling.shape[0], len(thatCOMID_sampling['date'].unique())]

    # Calculate IQR and median
    iqr = np.percentile(widths, 75) - np.percentile(widths, 25)
    median_widths = np.median(widths)
    count = thatCOMID_sampling.shape[0]
    days = len(thatCOMID_sampling['date'].unique())

    # Return the results for this COMID
    return [thatCOMID, iqr, median_widths, count, days]

# Define a helper function to pass to multiprocessing.Pool
def worker(job):
    return process_comid(*job)

In [17]:
# Create the job list (pairs of thatCOMID and thatCOMID_sampling)
jobs = [(thatCOMID, thatCOMID_sampling) for thatCOMID, thatCOMID_sampling in final_large_data_warm_over60.groupby('COMID')]

# Use multiprocessing with 6 cores and tqdm for progress
num_cores = 8
with multiprocessing.Pool(num_cores) as pool:
    results = list(tqdm(pool.imap(worker, jobs), total=len(jobs)))

# Convert the list of results to a DataFrame
results_df = pd.DataFrame(results, columns=['COMID', 'IQR', 'Median_Width', 'Count', 'Days'])

# Add a new column for IQRc
results_df['IQRc'] = results_df['IQR'] / results_df['Median_Width']

# Ensure Days and Count columns are integers
results_df['Days'] = results_df['Days'].astype(int)
results_df['Count'] = results_df['Count'].astype(int)

100%|████████████████████████████████| 476232/476232 [08:50<00:00, 897.93it/s]


In [18]:
output_path = 'NCDF_out/IQR_newresult_below_10m_removed.csv'
results_df.to_csv(output_path, index=False)

## Kruskal-Wallis test

In [20]:
from scipy.stats import kruskal

all_params = {}

min_data_taken_from_high_freq_stn = 0.6
base_date = pd.to_datetime('2000-01-01')
final_large_data_warm_over60['days2000'] = (final_large_data_warm_over60['date'] - base_date).dt.days
final_large_data_warm_over60['days2000'] = pd.to_datetime(final_large_data_warm_over60['date'].dt.strftime('%Y-%m-%d'))
latlon_comid = final_large_data_warm_over60.groupby('COMID').first().reset_index()


min_days_data = 20
min_data_taken_from_high_freq_stn = 0.6 #60% of the data is held

df_summary = pd.DataFrame(columns=['COMID', 'seasonality'])

count = 0
count_lake = 0
for thatCOMID, thatCOMID_sampling in tqdm(final_large_data_warm_over60.groupby(['COMID'])):
    count += 1
    
    river_id_counts = thatCOMID_sampling['riverID'].value_counts()
    cumulative_sum = river_id_counts.cumsum()
    total_sum = river_id_counts.sum()
    threshold = min_data_taken_from_high_freq_stn * total_sum
    filtered_data = thatCOMID_sampling[thatCOMID_sampling['riverID'].isin(river_id_counts.index[0:cumulative_sum[cumulative_sum < threshold].count() + 1])]
    
    filtered_data['width_normalized'] = filtered_data['width'] / filtered_data.groupby('riverID')['width'].transform('mean')
    median_days2000 = filtered_data.groupby('days2000')['width_normalized'].median()
    
    median = median_days2000.median()
    iqr = median_days2000.quantile(0.75) - median_days2000.quantile(0.25)
    
    threshold = 1.5 * iqr
    
    lower_bound = median - threshold
    upper_bound = median + threshold
    
    median_days2000 = median_days2000[(median_days2000 >= lower_bound) & (median_days2000 <= upper_bound)]
    
    if median_days2000.shape[0] < min_days_data:
        df_summary.loc[len(df_summary)] = [thatCOMID[0], 'low data']

        # all_params.append(("low data", thatCOMID, median_days2000.shape[0]))
        # all_params[thatCOMID] = ("low data", median_days2000.shape[0])
        continue
    
    dates = median_days2000.index
    widths = median_days2000.values

    data = {
        'datetime': dates,
        'data': widths
    }
    
    df = pd.DataFrame(data)
    df['month'] = df['datetime'].dt.month

    try:
        groups = [group['data'].values for name, group in df.groupby('month')]
        stat, p_value = kruskal(*groups)
        
        alpha = 0.05
        if p_value < alpha:
            df_summary.loc[len(df_summary)] = [thatCOMID[0], 'seasonal']
        else:
            df_summary.loc[len(df_summary)] = [thatCOMID[0], 'non seasonal']

    except ValueError as e:
        count_lake += 1
        df_summary.loc[len(df_summary)] = [thatCOMID[0], 'possible lake']

print(count)
print('Number of lakes are ', count_lake)


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 476232/476232 [1:20:25<00:00, 98.69it/s]


476232
Number of lakes are  165


In [21]:
output_path = 'NCDF_out/Kruskal_newresult_below_10m_removed.csv'
df_summary.to_csv(output_path, index=False)

## Peak month estimation

In [23]:
# Direct peak month estimation
def find_peak_months(df):
    peak_months = []
        
    # Extract year and month
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    
    # Group by year and month, averaging widths for each month
    monthly_data = df.groupby(['year', 'month'])['width'].max().reset_index()    ## here 'mean' was used earlier to compute monthly average widths
    
    # Check if at least 8 months of data are present per year
    #valid_years = monthly_data.groupby('year').filter(lambda x: len(x) >= 4)

    #if valid_years.shape[0] == 0: return "never 4 months"
    #if valid_years.shape[0] == 1: return "just 1 year"
    
    # Find the peak month for each year
    for year, group in monthly_data.groupby('year'):
        peak_month = group.loc[group['width'].idxmax()]['month']
        peak_months.append((year, peak_month))

    return peak_months

from collections import Counter

def filter_distant_months(most_common_months, threshold=1):
    """
    Removes months that are outside the ±threshold range from the most common month.
    """
    top_month, _ = most_common_months[0]
    filtered_months = []

    # Keep months that are within the threshold proximity of the top month
    for month, count in most_common_months:
        if abs(top_month - month) <= threshold or abs((top_month + 12) - month) <= threshold:
            filtered_months.append((month, count))
    
    return filtered_months
from collections import Counter

def determine_peak_month_with_strict_removal(peak_months, proximity_threshold=2):

    if peak_months == "never 8 months": return  "never 8 months"
    
    # Step 1: Count the frequency of each month
    month_counts = Counter([month for year, month in peak_months])
    
    # Step 2: Sort by frequency
    most_common_months = month_counts.most_common()
    
    # Step 3: Identify the most common (peak) month
    if len(most_common_months) > 1:

        peak_month = int(np.mean(most_common_months[0]))
        
        #peak_month = most_common_months[0][0]  # The top peak month
        
        # Step 4: Ensure all months are within ±2 month of the peak month
        #for month, count in most_common_months:
        #    if abs(peak_month - month) > proximity_threshold and abs((peak_month + 12) - month) > proximity_threshold:
        #        return "No clear peak"
    else:
        peak_month = most_common_months[0][0]

    if peak_months == "just 1 year": return "just 1 year " + str(peak_month)
    
    return peak_month

In [24]:
peak_months_list = []

for comid_id, comid_data in tqdm(final_large_data_warm_over60.groupby('COMID')):
    peak_months = find_peak_months(comid_data)  # Function assumes list input
    peak_month_result = determine_peak_month_with_strict_removal(peak_months)
    peak_months_list.append((comid_id, peak_month_result))

result_direct_peak_df = pd.DataFrame(peak_months_list, columns=['COMID', 'Direct Peak Month'])

Exception ignored in: <function tqdm.__del__ at 0x7fb005ade7a0>
Traceback (most recent call last):
  File "/N/lustre/project/proj-212/abhinav/environment/glows-venv/lib/python3.10/site-packages/tqdm/std.py", line 1148, in __del__
    self.close()
  File "/N/lustre/project/proj-212/abhinav/environment/glows-venv/lib/python3.10/site-packages/tqdm/std.py", line 1267, in close
    if self.disable:
AttributeError: 'tqdm' object has no attribute 'disable'
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 476232/476232 [20:10<00:00, 393.32it/s]


In [25]:
result_direct_peak_df.shape

(476232, 2)

In [27]:
result_direct_peak_df

,COMID,Direct Peak Month
0,11000054,4.0
1,11000302,1.0
2,11000308,7.0
3,11000309,2.0
4,11000766,6.0
...,...,...
476227,86007242,5.0
476228,86007254,5.0
476229,86007258,4.0
476230,86007267,6.0


In [28]:
output_path = 'NCDF_out/direct_peak newresult_below_10m_removed.csv'
result_direct_peak_df.to_csv(output_path, index=False)

In [11]:
result_direct_peak_df_numeric = result_direct_peak_df[
    pd.to_numeric(result_direct_peak_df['Direct Peak Month'], errors='coerce').notna()
]